# Running Eval14/15 in a QCrBox Docker container
Eval15 is a program to integrate crystallographic frames and evaluate the intensities of the reflections. The executable must be obtained separately from the [Eval website](http://www.crystal.chem.uu.nl/distr/eval/).

The example starts again by using the dotenv package. It makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, comment out the load_dotenv lines and insert the local and qcrbox pathes manually into the `pathhelper` using the information in your .env.dev file.

In [1]:
import os
import zipfile

from dotenv import load_dotenv

from qcrbox_wrapper import QCrBoxWrapper, QCrBoxPathHelper


if not load_dotenv('../.env.dev'):
    raise FileNotFoundError(
        ".dot.env file could not be loaded. Either adapt the path to your filesystem or "
        + "input the information loaded from os.environ manually"
    )


We create an example folder to try out the Eval functionality.

In [2]:
pathhelper = QCrBoxPathHelper(
    os.environ['QCRBOX_SHARED_FILES_DIR_HOST_PATH'],
    os.environ['QCRBOX_SHARED_FILES_DIR_CONTAINER_PATH'],
    'examples_eval'
)

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

frames_zip = './input_files/Ylid_OD_Images.zip'


## Connecting to QCrBox
We can connect to the QCrBox Inventory via python after we have started everything with qcb up. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [3]:
gui_infos = {'Eval1X': {'port': 12005, 'commands': ['interactive']}}
qcrbox = QCrBoxWrapper('localhost', 11000, gui_infos)

In [4]:
qcrbox.application_dict

{'Eval1X': Eval1X()}

Check that Eval1X is available in the application dict you just evaluated and then select it in the next cell.

In [5]:
eval1x = qcrbox.application_dict['Eval1X']


As always we can get the list of available commands using pythons buildin help function.

In [6]:
help(eval1x)

Help on QCrBoxApplication in module qcrbox_wrapper:

Eval1X()
    Represents the Eval1X application (v. 20230913) in QCrBox
    
    Methods:
        interactive(
            folder: str
        )
    
        integrate(
            work_folder: str,
            rmat_file_path: str,
            beamstop_file_path: str,
            detalign_file_path: str,
            maximum_res: str,
            minimum_res: str,
            box_size: str,
            box_depth: str,
            maximum_duration: str,
            min_refln_in_box: str,
            pic_dir: str
        )
    
        toparams__interactive(
            work_folder: str,
            par_json: str,
            par_folder: str
        )
    
        redo__interactive(
            work_folder: str,
            par_json: str,
            par_folder: str
        )
    
        interactive(
            work_folder: str
        )



## Starting an interactive session.
Currently the implementation of eval within QCrBox assumes that you first integrate your frames interactively following along [the thaumatin example from the Eval website](http://www.crystal.chem.uu.nl/distr/eval/documentation/ccd/doclib/example/thaumatin/index.html). 

A set of frames to use as example is included in `input_files/Ylid_OD_Images.zip`. So let us create a new folder and unpack the images to that folder.

In [7]:
# create a new folder for this example
folder_interact = path_local / 'run_interactive'
folder_interact.mkdir(exist_ok=True)

with zipfile.ZipFile(frames_zip, 'r') as zip_obj:
    zip_obj.extractall(folder_interact)

Now let us start an interactive session. Once we run the `interactive` command, two things happen: Firstly, a new browser window with the opens up, where a command line interface opens in the folder of your choice. Secondly, you should see an input prompt in your execution engine for this notebook. (In Jupyter this should be at the end of the cell, in VSCode it is at the top). By pressing enter within this prompt we tell QCrBox that we are done following the example. Subsequently QCrBox will try to run a few programs to produce a cif file with the output via Evals output to SADABS' .sad format. There is no cif file at the beginning of an integration. As such there cannot be an `input_cif_path`. Instead we need to pass the folder where we want to work that contains the frames. You can now try to follow along the [the thaumatin example from the Eval website](http://www.crystal.chem.uu.nl/distr/eval/documentation/ccd/doclib/example/thaumatin/index.html) using the Ylid frames.

Here are the changes to the example:

#### proteinsetup
The unit cell of Ylid is about a=5 Ang, b=8 Ang, c=18 Ang, change the maximum volume to 1500 Ang.

#### renameimages
The images are already renamed so skip

#### scancheck
You need to run `scandb` first

#### view
Do not change the detectoroffset from the values in the frames. The beamstop settings are 
 - `beamstop 0.0 -1.3`
 - `beamstopwidth 1.5`
 - `beamstopdiameter 4`
 - `beamstopangle 1`

Also set `resomax 0.79`, the number of peaks on a frame is 10

#### buildsearch 
Be sure to set the maximum resolution and the number of peaks per frame correctly

#### buildeval15
Use focus type mirror and no polarisation

#### shellscript-2
Use eval15all instead

#### any
You do not need to set the pointgroup with pg

#### sadabs and shellscript-3
sadabs is not independent of eval and therefore not available within the container.

In [8]:
calc1 = eval1x.interactive(path_qcrbox / 'run_interactive')

# Do not forget to press <enter> in the input prompt at the end!


KeyboardInterrupt: 

## Running an integration with slightly changed parameters

There is a basic way to run parts the data processing pipeline from the input files of an interactive session. The number of necessary files might be reduced in the future. This allows to reprocess frames that have been subject to Eval15 before. We can pull the necessary files from our interactive session from before to rerun the integration with a different resolution. Alternatively you can use the files in the `./input_files/eval_input.zip`. Let us create a folder again. You can set `use_files_from_zip` to `True` to use these.

In [ ]:
use_files_from_zip = False  # Change to True to use settings files from zip

# create a new folder for this example
folder_integrate = path_local / 'run_integrate'
folder_integrate.mkdir(exist_ok=True)

with zipfile.ZipFile(frames_zip, 'r') as zip_obj:
    zip_obj.extractall(folder_integrate)

if use_files_from_zip:
    with zipfile.ZipFile('./input_files/eval_input.zip', 'r') as zip_obj:
        zip_obj.extractall(folder_integrate)
    source_folder = path_qcrbox / 'run_integrate'
else:
    source_folder = path_qcrbox / 'run_interactive'

We can now start the integration using the output files from either the zip archive or the interactive session. You might notice that we pass a path to a cif file as the `rmat_file_path` if it contains the required entries. An rmat file will work as well. The integration will probably take a bit of time. You can follow the progress within the `ic` folder.

In [ ]:
calc2 = eval1x.integrate(
    work_folder=path_qcrbox / 'run_integrate',
    rmat_file_path=source_folder / 'output.cif',
    beamstop_file_path=source_folder / 'beamstop.vic',
    detalign_file_path=source_folder / 'detalign.vic',
    maximum_res=0.79,
    minimum_res=50.0,
    box_size=1.2,
    box_depth=5,
    maximum_duration=5.0,
    min_refln_in_box=1000,
    pic_dir=source_folder / 'ic'
)

calc2.wait_while_running(1.0)